<!-- Codes by HTMLcodes.ws -->
<h1 style = "color:Blue;font-family:newtimeroman;font-size:250%;text-align:center;border-radius:15px 50px;">Chatbot for Mental Health Conversations</h1>


In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

#from google.colab import drive
#drive.mount('/content/drive')

import os
for dirname, _, filenames in os.walk('.'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [2]:
import json

with open('intents.json', 'r') as f:
#with open('/content/drive/MyDrive/Apuntes/DASI/data_Jesus.json', 'r') as f:
    data = json.load(f)

init_df = pd.DataFrame(data['intents'])
# init_df = pd.DataFrame(data)
init_df

,tag,patterns,responses
0,greeting,"[Hi, Hey, Is anyone there?, Hi there, Hello, H...",[Hello there. Tell me how are you feeling toda...
1,morning,[Good morning],[Good morning. I hope you had a good night's s...
2,afternoon,[Good afternoon],[Good afternoon. How is your day going?]
3,evening,[Good evening],[Good evening. How has your day been?]
4,night,[Good night],"[Good night. Get some proper sleep, Good night..."
...,...,...,...
72,fact-28,[What do I do if I'm worried about my mental h...,[The most important thing is to talk to someon...
73,fact-29,[How do I know if I'm unwell?],"[If your beliefs , thoughts , feelings or beha..."
74,fact-30,[How can I maintain social connections? What i...,"[A lot of people are alone right now, but we d..."
75,fact-31,[What's the difference between anxiety and str...,[Stress and anxiety are often used interchange...


In [3]:
dic = {"tag":[], "patterns":[], "responses":[]}
for i in range(len(init_df)):
    ptrns = init_df[init_df.index == i]['patterns'].values[0]
    rspns = init_df[init_df.index == i]['responses'].values[0]
    tag = init_df[init_df.index == i]['tag'].values[0]
    for j in range(len(ptrns)):
        dic['tag'].append(tag)
        dic['patterns'].append(ptrns[j])
        dic['responses'].append(rspns)

df = pd.DataFrame.from_dict(dic)
df

,tag,patterns,responses
0,greeting,Hi,[Hello there. Tell me how are you feeling toda...
1,greeting,Hey,[Hello there. Tell me how are you feeling toda...
2,greeting,Is anyone there?,[Hello there. Tell me how are you feeling toda...
3,greeting,Hi there,[Hello there. Tell me how are you feeling toda...
4,greeting,Hello,[Hello there. Tell me how are you feeling toda...
...,...,...,...
226,fact-29,How do I know if I'm unwell?,"[If your beliefs , thoughts , feelings or beha..."
227,fact-30,How can I maintain social connections? What if...,"[A lot of people are alone right now, but we d..."
228,fact-31,What's the difference between anxiety and stress?,[Stress and anxiety are often used interchange...
229,fact-32,What's the difference between sadness and depr...,"[Sadness is a normal reaction to a loss, disap..."


In [4]:
df['tag'].unique()

array(['greeting', 'morning', 'afternoon', 'evening', 'night', 'goodbye',
       'thanks', 'no-response', 'neutral-response', 'about', 'skill',
       'creation', 'name', 'help', 'sad', 'stressed', 'worthless',
       'depressed', 'happy', 'casual', 'anxious', 'not-talking', 'sleep',
       'scared', 'death', 'understand', 'done', 'suicide', 'hate-you',
       'hate-me', 'default', 'jokes', 'repeat', 'wrong', 'stupid',
       'location', 'something-else', 'friends', 'ask', 'problem',
       'no-approach', 'learn-more', 'user-agree', 'meditation',
       'user-meditation', 'pandora-useful', 'user-advice',
       'learn-mental-health', 'mental-health-fact',
       'mental-health-definition', 'mental-health-importance',
       'depression-definition', 'depression-diagnostic',
       'therapist-definition', 'therapy-definition',
       'mental-illness-definition', 'mentall-illness-fact',
       'mental-illness-diagnostic', 'mental-illness-recovery',
       'mental-illness-helping', 'mental

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report
import plotly.graph_objects as go

In [8]:
# Split the dataset into training and testing sets
X = df['patterns']
y = df['tag']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
# Vectorize the text data using TF-IDF
vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

In [10]:
# Train a Support Vector Machine (SVM) classifier
model = SVC()
model.fit(X_train_vec, y_train)

SVC()

In [11]:
# Predict intents for the testing set
y_pred = model.predict(X_test_vec)

In [12]:
# Evaluate the model's performance
report = classification_report(y_test, y_pred, output_dict=True, zero_division=0)

In [13]:
# Convert float values in the report to dictionaries
report = {label: {metric: report[label][metric] for metric in report[label]} for label in report if isinstance(report[label], dict)}

In [14]:
# Extract evaluation metrics
labels = list(report.keys())
evaluation_metrics = ['precision', 'recall', 'f1-score']
metric_scores = {metric: [report[label][metric] for label in labels if label in report] for metric in evaluation_metrics}
print(metric_scores['precision'])
print(metric_scores['recall'])
print(metric_scores['f1-score'])

In [16]:
import random
# Prediction Model Deployment

# A trained SVM model named 'model' and a vectorizer named 'vectorizer'

# Function to predict intents based on user input
def predict_intent(user_input):
    # Vectorize the user input
    user_input_vec = vectorizer.transform([user_input])

    # Predict the intent
    #print(model.predict(user_input_vec))
    intent = model.predict(user_input_vec)[0]

    return intent


In [17]:
# Function to generate responses based on predicted intents
def generate_response(intent, df):
    # Implement your logic here to generate appropriate responses based on the predicted intents
    # if intent == 'greeting':
    #    response = "Hello! How can I assist you today?"
    # elif intent == 'farewell':
    #    response = "Goodbye! Take care."
    # elif intent == 'question':
    #    response = "I'm sorry, I don't have the information you're looking for."
    # else:
    #     response = "I'm here to help. Please let me know how I can assist you."
    if (intent in df['tag'].unique()):
        print(f"Intent '{intent}' found in tags")
        allResponses = df[df.tag == intent]['responses']
        index = allResponses.index[0]
        #print(allResponses.index)
        num_available_responses = len(allResponses[index])
        id_resp = random.randint(0, num_available_responses - 1)
        response = allResponses[index][id_resp]
    else:
        response = "I'm here to help. Please let me know how I can assist you."

    return response

In [ ]:
# Example usage
while True:
    # Get user input
    user_input = input("User: ")

    # Predict intent
    intent = predict_intent(user_input)

    # Generate response
    response = generate_response(intent, init_df)

    print("Chatbot:", response)


User:  Hi, I'm feeling depressed


greeting
Intent found in tags
5
0
Chatbot: Hello there. Tell me how are you feeling today?


User:  I've said I'm feeling depressed


greeting
Intent found in tags
5
3
Chatbot: Great to see you. How do you feel currently?


User:  I've feeling the need to jump through the window


greeting
Intent found in tags
5
1
Chatbot: Hi there. What brings you here today?
